In [20]:
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='arcticecosystems')

import pandas as pd

In [5]:
startDate = "2020-06-01"
advanceDays = 30
dateRange = ee.DateRange(ee.Date(startDate), ee.Date(startDate).advance(advanceDays, 'day'))
geometry = ee.Geometry.Point([-129.9930708040777, 68.59277781573002])

In [6]:
testLandsatImage = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').select(
    ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'ST_B10', 'QA_PIXEL'],
    ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'TIR', 'QA_PIXEL']
).filterBounds(geometry).filterDate(dateRange).filter(ee.Filter.lt('CLOUD_COVER', 10))

In [7]:
nbrExpression = '(b("SWIR1") - b("NIR")) / (b("SWIR1") + b("NIR"))'
nbr = testLandsatImage.first().expression(nbrExpression).rename('nbr')

ndviExpression = '(b("NIR") - b("Red")) / (b("NIR") + b("Red"))'
ndvi = testLandsatImage.first().expression(ndviExpression).rename('ndvi')

##mnliExpression = '( ( (b("NIR") * b("NIR")) - b("Red") ) * (1 + 0.5) ) / ( ( b("NIR") * b("NIR") ) + b("Red") + 0.5 )'
##mnli = testLandsatImage.first().expression(mnliExpression).rename('mnli')

saviExpression = '1.5 * (b("NIR") - b("Red")) / (b("NIR") + b("Red") + 0.5)'
savi = testLandsatImage.first().expression(saviExpression).rename('savi')

rdviExpression = '(b("NIR") - b("Red")) / sqrt((b("NIR") + b("Red")))'
rdvi = testLandsatImage.first().expression(rdviExpression).rename('rdvi')

gviExpression = '(-0.2848 * b("Blue")) + (-0.2435 * b("Green")) + (-0.5436 * b("Red")) + (0.7243 * b("NIR")) + (0.0840 * b("SWIR1")) + (-0.1800 * b("SWIR2"))'
gvi = testLandsatImage.first().expression(gviExpression).rename('gvi')

ndmiExpression = '(b("NIR") - b("SWIR1")) / (b("NIR") + b("SWIR1"))'
ndmi = testLandsatImage.first().expression(ndmiExpression).rename('ndmi')

ndwiExpression = '(b("Green") - b("SWIR1")) / (b("Green") + b("SWIR1"))'
ndwi = testLandsatImage.first().expression(ndwiExpression).rename('ndwi')

In [8]:
dataset = ee.ImageCollection('MODIS/061/MCD64A1').filter(ee.Filter.date('2000-11-01', '2024-11-01'))
burnedArea = dataset.select('BurnDate')

In [9]:
Map = geemap.Map()
Map.centerObject(testLandsatImage, zoom = 8)

Map.addLayer(testLandsatImage, {}, 'landsat')
Map.addLayer(nbr, {'min': 0, 'max': 0.2, 'palette': 'Reds'}, 'nbr')
Map.addLayer(ndvi, {'min': 0, 'max': 0.2, 'palette': 'Greens'}, 'ndvi', False)
Map.addLayer(savi, {'min': 0, 'max': 0.3, 'palette': 'Greens'}, 'savi')
Map.addLayer(rdvi, {'min': 4.3, 'max': 30.9, 'palette': 'Greens'}, 'rdvi')
Map.addLayer(gvi, {'min': -5253.5, 'max': 51.7, 'palette': 'Greens'}, 'gvi')
Map.addLayer(ndmi, {'min': -0.2, 'max': 0.1, 'palette': 'Blues'}, 'ndmi')
Map.addLayer(ndwi, {'min': -0.2, 'max': -0.1, 'palette': 'Purples'}, 'ndwi')
Map.addLayer(burnedArea, {}, 'modis precomp burned area', False)

Map

Map(center=[68.27931430329704, -132.89380539556925], controls=(WidgetControl(options=['position', 'transparent…

dNBR thresholds (https://www.earthdatascience.org/courses/earth-analytics/multispectral-remote-sensing-modis/normalized-burn-index-dNBR/)
SEVERITY LEVEL	 	dNBR RANGE
Enhanced Regrowth	 	< -.1
Unburned	 	-.1 to +.1
Low Severity	 	+.1 to +.27
Moderate Severity	 	+.27 to +.66
High Severity	 	> .66

In [10]:
nbrThresholdRegrowth = nbr.lte(-0.1).rename('nbrRegrowth')
nbrThresholdUnburned = nbr.gt(-0.1).And(nbr.lt(0.1)).rename('nbrUnburned')
nbrThresholdAllSeverity = nbr.gt(0.1).rename('allSeverity')
nbrThresholdLowSeverity = nbr.gt(0.1).And(nbr.lt(0.27)).rename('lowSeverity')
nbrThresholdMedSeverity = nbr.gt(0.27).And(nbr.lt(0.66)).rename('medSeverity')
nbrThresholdHiSeverity = nbr.gte(0.66).rename('hiSeverity')

In [11]:
Map = geemap.Map()
Map.centerObject(testLandsatImage, zoom = 8)
Map.addLayer(testLandsatImage, {}, 'landsat')
Map.addLayer(nbrThresholdRegrowth, {'palette': ['ffffff', 'fc4b05']}, 'nbrThresholdRegrowth', False)
Map.addLayer(nbrThresholdUnburned, {'palette': ['ffffff', 'fc4b05']}, 'nbrThresholdUnburned', False)
Map.addLayer(nbr.updateMask(nbrThresholdAllSeverity), {'palette': "Reds"}, 'nbrThresholdAllSeverity')
Map.addLayer(nbrThresholdLowSeverity, {'palette': ['ffffff', 'fc4b05']}, 'nbrThresholdLowSeverity', False)
Map.addLayer(nbrThresholdMedSeverity, {'palette': ['ffffff', 'fc4b05']}, 'nbrThresholdMedSeverity', False)
Map.addLayer(nbrThresholdHiSeverity, {'palette': ['ffffff', 'fc4b05']}, 'nbrThresholdHiSeverity', False)

Map


Map(center=[68.27931430329704, -132.89380539556925], controls=(WidgetControl(options=['position', 'transparent…

In [50]:
nbrAllSeverityMean = nbr.updateMask(nbrThresholdAllSeverity).reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': testLandsatImage.first().geometry(),
    'scale': 30,
    'bestEffort': True
})
##print(f'Mean nbr of image from all severity burn values: {nbrAllSeverityMean.getInfo()}')
print(nbrAllSeverityMean.keys().getInfo())
print(nbrAllSeverityMean.getInfo()['nbr'])
print(nbrAllSeverityMean.get('nbr').getInfo())

['nbr']
0.11931346127949093
0.11931346127949093


In [54]:
redRegOutputasFeature = ee.Feature(
    geom = testLandsatImage.first().geometry(),
    properties = nbrAllSeverityMean)
print(redRegOutputasFeature.getInfo())

{'type': 'Feature', 'geometry': {'type': 'Polygon', 'coordinates': [[[-135.87088137500578, 67.79675100172878], [-135.86955436703474, 67.79626298846752], [-135.68821273869753, 67.76922771572154], [-134.91625459716408, 67.65117936096587], [-132.5020514660815, 67.24991298399674], [-131.8350240185899, 67.13015192887212], [-131.45159430165663, 67.44420471710342], [-131.43082844583785, 67.46099290535561], [-130.96699491361787, 67.83010248427415], [-130.52896364730384, 68.16744941733839], [-130.31836089263246, 68.32588671504922], [-130.17715260561803, 68.43083865108512], [-129.83987485173483, 68.67793746173336], [-129.79314078444455, 68.71170923624955], [-129.79433241013638, 68.71222264601099], [-130.6516334182185, 68.8656411473199], [-133.11602649815362, 69.27197176585348], [-134.0203687943106, 69.40872432428796], [-134.07756644218276, 69.41714113658378], [-134.0826012555486, 69.41306287223404], [-135.87090872890104, 67.79749007405859], [-135.87088137500578, 67.79675100172878]]]}, 'propertie

In [29]:
testArray = ee.Array([[1,2,3,4,5], [6,7,8,9,0]])
print(testArray.getInfo())

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 0]]


In [24]:
def ee_array_to_df(arr, list_of_bands):
    """Transforms client-side ee.Image.getRegion array to pandas.DataFrame."""
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['time','datetime',  *list_of_bands]]

    return df